In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import sys
sys.path.append('../secretary')

import pandas as pd
from secretary import todo
import secretary.secretary_bot as sb
from io import StringIO


In [50]:
with open('legalese_email.txt', 'r') as file:
    content = file.read()

message = "From: Jack\n" + content

df_legal = sb.extract_tasks(message)
df_legal

,tags,type,due date,requestor,actor,summary,notes
0,"[probate, finance, creditor claims]",question,NaN,Marcella Brodhurst,Jack,Confirmation of payment to Truist Bank,"Have you made the payment to Truist? If so, ha..."
1,"[probate, estate management]",action_item,NaN,Marcella Brodhurst,Jack,Confirm removal of contents and closure of saf...,Please confirm that the contents have been rem...
2,"[probate, estate management]",action_item,NaN,Marcella Brodhurst,Jack,Provide update on account closures and funds,Mark wants to know: have you closed all the ac...
3,"[probate, finance]",question,NaN,Marcella Brodhurst,Jack,Inquire about income and interest on estate ac...,Can you tell us how much income and/or interes...
4,"[probate, taxes]",question,NaN,Marcella Brodhurst,Jack,Inquire about the final 1040 income tax return,Have you taken care of the final 1040 final in...


In [51]:
todos = todo.Todo()

tasks = [
  {
    "tags": ["shopping", "shoes"],
    "type": "action_item",
    "due date": "2024-04-14",
    "requestor": "Jack",
    "actor": "Jack",
    "summary": "Buy shoes",
    "notes": "I need to buy some cool new kicks."
  },
  {
    "tags": ["shopping", "groceries"],
    "type": "action_item",
    "due date": "2024-05-14",
    "requestor": "Jack",
    "actor": "Silvia",
    "summary": "Buy eggs",
    "notes": "Silvia needs to buy eggs"
  }
]

df = pd.concat([df_legal, pd.DataFrame(tasks)], ignore_index=True)
todos.add_new_task_to_database(df)
todos.df

,completed,tags,type,due date,requestor,actor,summary,notes,embedding
0,False,"[probate, finance, creditor claims]",question,NaN,Marcella Brodhurst,Jack,Confirmation of payment to Truist Bank,"Have you made the payment to Truist? If so, ha...","[0.013396532274782658, -0.011826315894722939, ..."
1,False,"[probate, estate management]",action_item,NaN,Marcella Brodhurst,Jack,Confirm removal of contents and closure of saf...,Please confirm that the contents have been rem...,"[0.022477947175502777, 0.05522691830992699, -0..."
2,False,"[probate, estate management]",action_item,NaN,Marcella Brodhurst,Jack,Provide update on account closures and funds,Mark wants to know: have you closed all the ac...,"[0.002025411231443286, 0.0706426352262497, -0...."
3,False,"[probate, finance]",question,NaN,Marcella Brodhurst,Jack,Inquire about income and interest on estate ac...,Can you tell us how much income and/or interes...,"[0.012339883483946323, 0.01436946913599968, 0...."
4,False,"[probate, taxes]",question,NaN,Marcella Brodhurst,Jack,Inquire about the final 1040 income tax return,Have you taken care of the final 1040 final in...,"[0.020119642838835716, 0.013605993241071701, 0..."
5,False,"[shopping, shoes]",action_item,2024-04-14,Jack,Jack,Buy shoes,I need to buy some cool new kicks.,"[-0.013687492348253727, -0.0008522969437763095..."
6,False,"[shopping, groceries]",action_item,2024-05-14,Jack,Silvia,Buy eggs,Silvia needs to buy eggs,"[-0.02773847058415413, -0.012552536092698574, ..."


In [43]:
comment = "From: Jack\nI still have my shitty old NewBalance sneakers from 2018, but I really do like them."

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.194925
1    0.176854
2    0.160165
3    0.114002
4    0.088483
5    0.163126
6    0.147721
7    0.191314
8    0.398913
9    0.188079
Name: embedding, dtype: float64
Similar task(s):
{
    "8":{
        "completed":false,
        "tags":[
            "shopping",
            "shoes"
        ],
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks."
    }
}
Updated task(s):
{
    "8":{
        "completed":false,
        "tags":[
            "shopping",
            "shoes"
        ],
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks. *I still have my shitty old NewBalance sneakers from 2018, but I really do like them.*"
    }
}


In [44]:
comment = 'From: Jack\nI bought some boots.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.262145
1    0.244156
2    0.279803
3    0.151544
4    0.165858
5    0.302728
6    0.240395
7    0.271184
8    0.525278
9    0.344048
Name: embedding, dtype: float64
Similar task(s):
{
    "5":{
        "completed":false,
        "tags":[
            "probate",
            "estate closing",
            "petition for discharge"
        ],
        "type":"action_item",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Marcella Brodhurst",
        "summary":"Prepare Petition for Discharge for Jack to sign",
        "notes":"We will send to you to sign at the appropriate time."
    },
    "8":{
        "completed":false,
        "tags":[
            "shopping",
            "shoes"
        ],
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks."
    },
    "9":{
        "completed":false,
 

In [45]:
comment = 'From: Jack\nI finished the task due on 04/14'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.354758
1    0.350331
2    0.388705
3    0.266684
4    0.282179
5    0.448206
6    0.355846
7    0.485911
8    0.409287
9    0.382457
Name: embedding, dtype: float64
Similar task(s):
{
    "0":{
        "completed":false,
        "tags":[
            "probate",
            "creditors",
            "payment",
            "Truist Bank"
        ],
        "type":"question",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Jack",
        "summary":"Confirmation of payment to Truist Bank",
        "notes":"Have you made the payment to Truist? If so, have they sent you a Satisfaction and Release of Claim? Please forward to us that was sent to you so we can file it."
    },
    "1":{
        "completed":false,
        "tags":[
            "probate",
            "creditors",
            "Truist Bank",
            "communication"
        ],
        "type":"action_item",
        "due date":null,
        "requestor":"Marcella Brodhurst",

In [46]:
comment = 'From: Jack\nActually, Nick should buy the eggs.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.224052
1    0.234461
2    0.286983
3    0.147512
4    0.144988
5    0.297688
6    0.254854
7    0.288175
8    0.369407
9    0.503712
Name: embedding, dtype: float64
Similar task(s):
{
    "8":{
        "completed":false,
        "tags":[
            "shopping",
            "shoes"
        ],
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks."
    },
    "9":{
        "completed":false,
        "tags":[
            "shopping",
            "groceries"
        ],
        "type":"action_item",
        "due date":"2024-05-14",
        "requestor":"Jack",
        "actor":"Silvia",
        "summary":"Buy eggs",
        "notes":"Silvia needs to buy eggs"
    }
}
Updated task(s):
{
    "9":{
        "completed":false,
        "tags":[
            "shopping",
            "groceries"
        ],
        "type":"action_item",
  

In [47]:
comment = 'From: Jack\nI just love zebras!'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.141180
1    0.107591
2    0.126674
3    0.056692
4    0.067624
5    0.178468
6    0.132066
7    0.173890
8    0.263863
9    0.168640
Name: embedding, dtype: float64
Similar task(s):
{

}
Updated task(s):
{

}


In [48]:
comment = 'From: Jack\nI need to hang out with Kodak soon.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.217239
1    0.214081
2    0.260996
3    0.154202
4    0.154029
5    0.332331
6    0.228016
7    0.278354
8    0.353220
9    0.288437
Name: embedding, dtype: float64
Similar task(s):
{
    "5":{
        "completed":false,
        "tags":[
            "probate",
            "estate closing",
            "petition for discharge"
        ],
        "type":"action_item",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Marcella Brodhurst",
        "summary":"Prepare Petition for Discharge for Jack to sign",
        "notes":"We will send to you to sign at the appropriate time."
    },
    "8":{
        "completed":false,
        "tags":[
            "shopping",
            "shoes"
        ],
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks."
    }
}
Updated task(s):
{

}


In [49]:
comment = 'From: Jack\nMerge tasks 2 and 3 about the probate process estate accounts.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.456467
1    0.465479
2    0.507061
3    0.437924
4    0.485072
5    0.569186
6    0.556297
7    0.512728
8    0.287101
9    0.272436
Name: embedding, dtype: float64
Similar task(s):
{
    "0":{
        "completed":false,
        "tags":[
            "probate",
            "creditors",
            "payment",
            "Truist Bank"
        ],
        "type":"question",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Jack",
        "summary":"Confirmation of payment to Truist Bank",
        "notes":"Have you made the payment to Truist? If so, have they sent you a Satisfaction and Release of Claim? Please forward to us that was sent to you so we can file it."
    },
    "1":{
        "completed":false,
        "tags":[
            "probate",
            "creditors",
            "Truist Bank",
            "communication"
        ],
        "type":"action_item",
        "due date":null,
        "requestor":"Marcella Brodhurst",